In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd


In [ ]:
credits=pd.read_csv('/kaggle/input/credits-data/tmdb_5000_credits.csv')
movies=pd.read_csv('/kaggle/input/movies-data/tmdb_5000_movies.csv')

In [ ]:
movies.head(1)

In [ ]:
credits.head(1)

In [ ]:
movies=movies.merge(credits,on='title')

In [ ]:
movies['original_language'].value_counts()

In [ ]:
movies.shape

In [ ]:
movies.head(1)

In [ ]:
#genres
#id
#keywords
#title
#overview
#cast
#crew

movies=movies[['id','title','overview','genres','keywords','cast','crew']]

In [ ]:
movies.info()

In [ ]:
movies.head()

In [ ]:
movies.isnull().sum()

In [ ]:
movies.dropna(inplace=True)

In [ ]:
movies.duplicated().sum()

In [ ]:
movies.iloc[0].genres

In [ ]:
import ast

In [ ]:
#['Action','Adventures','Scifi']

def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L    

In [ ]:
convert('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

In [ ]:

ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

In [ ]:
movies['genres']=movies['genres'].apply(convert)

In [ ]:
movies.head()

In [ ]:
movies['keywords']=movies['keywords'].apply(convert)

In [ ]:
movies.head()

In [ ]:
def convert3(obj):
    L=[]
    count=0
    for i in ast.literal_eval(obj):
        if count!=3:
            L.append(i['name'])
            count+=1
        else:
            break
    return L    

In [ ]:
movies['cast']=movies['cast'].apply(convert3)

In [ ]:
movies.head()

In [ ]:
movies['crew'][0]

In [ ]:
def fetch_director(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            L.append(i['name'])
            break      
    return L  

In [ ]:
movies['crew']=movies['crew'].apply(fetch_director)

In [ ]:
movies.head()

In [ ]:
movies['overview'][0]

In [ ]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [ ]:
movies.head()

In [ ]:
#Science Fiction = ScienceFiction

movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies.head()

In [ ]:
movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [ ]:
movies.head()

In [ ]:
new_df=movies[['id','title','tags']]

In [ ]:
new_df

In [ ]:
new_df['tags']=new_df['tags'].apply(lambda x: " ".join(x))

In [ ]:
new_df['tags'][0]

In [ ]:
new_df.head()

In [ ]:
#stemming 
#[loving,loved,love] converts into [love,love,love]
import nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [ ]:
def stem(text):
    y=[]
    
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)    

In [ ]:
new_df['tags']=new_df['tags'].apply(stem)

In [ ]:
new_df['tags']=new_df['tags'].apply(lambda x:x.lower())

In [ ]:
new_df.head(1)

In [ ]:
#Vectorization
#bag of words
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')


In [ ]:
vectors=cv.fit_transform(new_df['tags']).toarray()

In [ ]:
vectors

In [ ]:
cv.get_feature_names()

In [ ]:
ps.stem('danceing')

In [ ]:
stem('In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver JamesCameron')

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity=cosine_similarity(vectors)

In [ ]:
similarity.shape

In [ ]:
similarity[0]

In [ ]:
new_df[new_df['title']=='Avatar'].index[0]

In [ ]:
#Model

def recommend(movie):
    movie_index=new_df[new_df['title']==movie].index[0]
    distances=similarity[movie_index]
    movies_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    
    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [ ]:
recommend('Avatar')

In [ ]:
new_df.iloc[1216].title

In [ ]:
import pickle

In [ ]:
pickle.dump(new_df,open('movies.pkl','wb'))

In [ ]:
pickle.dump(new_df.to_dict(),open('movies_dict.pkl','wb'))

In [ ]:
new_df['title'].values

In [ ]:
pickle.dump(similarity,open('similarity.pkl','wb'))